### Datasets
http://www.manythings.org/anki/  (Download and unzip mar-eng.zip file)

In [1]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

Using TensorFlow backend.


In [2]:
#data_path = "\\NMT\\hin-eng\\hin.txt"
data_path = "NMT\\spa-eng\\spa.txt"

In [3]:
lines= pd.read_table(data_path,  names =['source', 'target', 'comments'])

In [4]:
lines.shape

(122936, 3)

In [5]:
lines[:10]

,source,target,comments
0,Go.,Ve.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1,Go.,Vete.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
2,Go.,Vaya.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
3,Go.,Váyase.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
4,Hi.,Hola.,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
5,Run!,¡Corre!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
6,Run!,¡Corran!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
7,Run!,¡Corra!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
8,Run!,¡Corred!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
9,Run.,Corred.,CC-BY 2.0 (France) Attribution: tatoeba.org #4...


In [6]:
lines.sample(10)

,source,target,comments
63975,My companions were all asleep.,Mis compañeros estaban todos dormidos.,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
78073,Do you really want to get married?,¿De verdad te quieres casar?,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
109420,This science-fiction novel is very interesting.,Esta novela de ciencia ficción es muy interesa...,CC-BY 2.0 (France) Attribution: tatoeba.org #6...
64049,No one lives in this building.,Nadie vive en este edificio.,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
23291,Who owns this truck?,¿Quién es el propietario de este camión?,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
99260,There is no reason why I shouldn't do it.,No hay una razón para no hacerlo.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
95258,What do you usually have for breakfast?,¿Qué tomas para desayunar?,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
110530,That outfit is running a fly-by-night operation.,Ese equipo está realizando una operación sospe...,CC-BY 2.0 (France) Attribution: tatoeba.org #6...
74404,God created man in his own image.,Dios creó al hombre a su imagen.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
103187,This is the restaurant that I often eat at.,Éste es el restaurante en el que suelo comer.,CC-BY 2.0 (France) Attribution: tatoeba.org #6...


In [7]:
# Lowercase all characters
lines.source=lines.source.apply(lambda x: x.lower())
lines.target=lines.target.apply(lambda x: x.lower())

In [8]:
# Remove quotes
lines.source=lines.source.apply(lambda x: re.sub("'", '', x))
lines.targetr=lines.target.apply(lambda x: re.sub("'", '', x))

C:\Users\khandelwalr\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines.source=lines.source.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.target=lines.target.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [10]:
print(exclude)

{'/', ':', '<', '^', ',', '}', '_', '&', '{', '#', '+', '?', '|', '%', '(', '~', '!', '=', "'", ';', ')', '`', '>', '-', '$', '"', '\\', '.', ']', '[', '*', '@'}


In [11]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
print(remove_digits)
lines.source=lines.source.apply(lambda x: x.translate(remove_digits))
lines.target=lines.target.apply(lambda x: x.translate(remove_digits))
#lines.target = lines.target.apply(lambda x: re.sub("[0१२३०८५७९४६]", "", x))

{48: None, 49: None, 50: None, 51: None, 52: None, 53: None, 54: None, 55: None, 56: None, 57: None}


In [12]:
# Remove extra spaces
lines.source=lines.source.apply(lambda x: x.strip())
lines.target=lines.target.apply(lambda x: x.strip())
lines.source=lines.source.apply(lambda x: re.sub(" +", " ", x))
lines.target=lines.target.apply(lambda x: re.sub(" +", " ", x))

In [13]:
# Add start and end tokens to target sequences
lines.target = lines.target.apply(lambda x : 'START_ '+ x + ' _END')

In [14]:
lines.sample(10)

,source,target,comments
12926,you deserve more,START_ tú mereces más _END,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
83721,to make matters worse he fell ill,START_ para empeorar las cosas se enfermó _END,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
25922,that wasnt my fault,START_ eso no fue mi culpa _END,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
40788,everything is going well,START_ todo va bien _END,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
121268,it is stipulated that we should receive a mont...,START_ está estipulado que deberíamos recibir ...,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
116570,there must be a way we can convince tom to wor...,START_ debe haber una manera de convencer a to...,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
7829,come and see me,START_ ven a verme _END,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
78151,einstein liked to play the violin,START_ a einstein le gustaba tocar el violín _END,CC-BY 2.0 (France) Attribution: tatoeba.org #6...
85439,i get out of the hospital next week,START_ salgo del hospital la semana que viene ...,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
85700,i was concerned for obvious reasons,START_ estaba preocupado por razones evidentes...,CC-BY 2.0 (France) Attribution: tatoeba.org #6...


In [15]:
# Vocabulary of Source language
all_source_words=set()
for source in lines.source:
    for word in source.split():
        if word not in all_source_words:
            all_source_words.add(word)

# Vocabulary of Target 
all_target_words=set()
for target in lines.target:
    for word in target.split():
        if word not in all_target_words:
            all_target_words.add(word)


In [16]:
print(len(all_source_words))

13467


In [17]:
input_words = sorted(list(all_source_words))
target_words = sorted(list(all_target_words))
input_words

['a',
 'aardvark',
 'aardvarks',
 'aaron',
 'aback',
 'abandon',
 'abandoned',
 'abandoning',
 'abate',
 'abated',
 'abating',
 'abbreviation',
 'abc',
 'abdicate',
 'abdomen',
 'abdominal',
 'abducted',
 'aberration',
 'abhor',
 'abhorrent',
 'abide',
 'abiding',
 'abilities',
 'ability',
 'able',
 'abnormal',
 'aboard',
 'abolish',
 'abolished',
 'aboriginal',
 'about',
 'above',
 'abraham',
 'abreast',
 'abridged',
 'abroad',
 'abrupt',
 'absence',
 'absences',
 'absent',
 'absentminded',
 'absentmindedly',
 'absolute',
 'absolutely',
 'absorb',
 'absorbed',
 'absorbs',
 'abstain',
 'abstained',
 'abstract',
 'absurd',
 'absurdly',
 'abundant',
 'abuse',
 'abused',
 'abuses',
 'abusing',
 'academic',
 'academy',
 'accelerated',
 'accent',
 'accept',
 'acceptable',
 'accepted',
 'accepting',
 'accepts',
 'access',
 'accessible',
 'accessories',
 'accident',
 'accidental',
 'accidentally',
 'accidents',
 'accommodate',
 'accommodation',
 'accompanied',
 'accompany',
 'accomplices',
 '

In [18]:
# Max Length of source sequence
lenght_list=[]
for l in lines.source:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)
max_length_src

47

In [19]:
# Max Length of target sequence
lenght_list=[]
for l in lines.target:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
max_length_tar

55

In [20]:
# Max Length of source sequence
lenght_list=[]
for l in lines.source:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)
print(" Max length of the source sentence", max_length_src)
# Max Length of target sequence
lenght_list=[]
for l in lines.target:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
print(" Max length of the target sentence",max_length_tar)

 Max length of the source sentence 47
 Max length of the target sentence 55


In [21]:
num_encoder_tokens = len(all_source_words)
num_decoder_tokens = len(all_target_words)
num_encoder_tokens, num_decoder_tokens

(13467, 27199)

In [22]:
num_decoder_tokens += 1 # For zero padding
num_decoder_tokens

27200

Thehttps://github.com/keras-team/keras/issues/2375 padding is useful when you batch your sequences. If you don't want to mask, you have several options:

1 - batch_size=1. You feed the sequences one by one and in this case you don't need to have them of the same length. Something like (from my memory):The padding is useful when you batch your sequences. If you don't want to mask, you have several options:

1 - batch_size=1. You feed the sequences one by one and in this case you don't need to have them of the same length. Something like (from my memory):https://github.com/keras-team/keras/issues/2375https://github.com/keras-team/keras/issues/2375


In [23]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])
print(input_token_index)

{'a': 1, 'aardvark': 2, 'aardvarks': 3, 'aaron': 4, 'aback': 5, 'abandon': 6, 'abandoned': 7, 'abandoning': 8, 'abate': 9, 'abated': 10, 'abating': 11, 'abbreviation': 12, 'abc': 13, 'abdicate': 14, 'abdomen': 15, 'abdominal': 16, 'abducted': 17, 'aberration': 18, 'abhor': 19, 'abhorrent': 20, 'abide': 21, 'abiding': 22, 'abilities': 23, 'ability': 24, 'able': 25, 'abnormal': 26, 'aboard': 27, 'abolish': 28, 'abolished': 29, 'aboriginal': 30, 'about': 31, 'above': 32, 'abraham': 33, 'abreast': 34, 'abridged': 35, 'abroad': 36, 'abrupt': 37, 'absence': 38, 'absences': 39, 'absent': 40, 'absentminded': 41, 'absentmindedly': 42, 'absolute': 43, 'absolutely': 44, 'absorb': 45, 'absorbed': 46, 'absorbs': 47, 'abstain': 48, 'abstained': 49, 'abstract': 50, 'absurd': 51, 'absurdly': 52, 'abundant': 53, 'abuse': 54, 'abused': 55, 'abuses': 56, 'abusing': 57, 'academic': 58, 'academy': 59, 'accelerated': 60, 'accent': 61, 'accept': 62, 'acceptable': 63, 'accepted': 64, 'accepting': 65, 'accepts

In [24]:
print(target_token_index)

{'START_': 1, '_END': 2, 'a': 3, 'aabe': 4, 'aah': 5, 'aaron': 6, 'abajo': 7, 'abandona': 8, 'abandonada': 9, 'abandonadas': 10, 'abandonado': 11, 'abandonados': 12, 'abandonamos': 13, 'abandonan': 14, 'abandonar': 15, 'abandonaremos': 16, 'abandonarlo': 17, 'abandonarlos': 18, 'abandonaron': 19, 'abandonarte': 20, 'abandonará': 21, 'abandonaré': 22, 'abandonaría': 23, 'abandonarías': 24, 'abandonas': 25, 'abandonase': 26, 'abandonaste': 27, 'abandone': 28, 'abandonen': 29, 'abandono': 30, 'abandoné': 31, 'abandonó': 32, 'abandónalo': 33, 'abanicando': 34, 'abanico': 35, 'abarca': 36, 'abarrotada': 37, 'abarrotadas': 38, 'abarrotado': 39, 'abarrotes': 40, 'abastece': 41, 'abastecida': 42, 'abastecido': 43, 'abatido': 44, 'abatió': 45, 'abdicar': 46, 'abdicó': 47, 'abdomen': 48, 'abdominales': 49, 'abducida': 50, 'abducido': 51, 'abducidos': 52, 'abecedario': 53, 'abeja': 54, 'abejas': 55, 'aberración': 56, 'abeto': 57, 'abetos': 58, 'abierta': 59, 'abiertas': 60, 'abierto': 61, 'abiert

In [25]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [26]:
print(reverse_input_char_index)

{1: 'a', 2: 'aardvark', 3: 'aardvarks', 4: 'aaron', 5: 'aback', 6: 'abandon', 7: 'abandoned', 8: 'abandoning', 9: 'abate', 10: 'abated', 11: 'abating', 12: 'abbreviation', 13: 'abc', 14: 'abdicate', 15: 'abdomen', 16: 'abdominal', 17: 'abducted', 18: 'aberration', 19: 'abhor', 20: 'abhorrent', 21: 'abide', 22: 'abiding', 23: 'abilities', 24: 'ability', 25: 'able', 26: 'abnormal', 27: 'aboard', 28: 'abolish', 29: 'abolished', 30: 'aboriginal', 31: 'about', 32: 'above', 33: 'abraham', 34: 'abreast', 35: 'abridged', 36: 'abroad', 37: 'abrupt', 38: 'absence', 39: 'absences', 40: 'absent', 41: 'absentminded', 42: 'absentmindedly', 43: 'absolute', 44: 'absolutely', 45: 'absorb', 46: 'absorbed', 47: 'absorbs', 48: 'abstain', 49: 'abstained', 50: 'abstract', 51: 'absurd', 52: 'absurdly', 53: 'abundant', 54: 'abuse', 55: 'abused', 56: 'abuses', 57: 'abusing', 58: 'academic', 59: 'academy', 60: 'accelerated', 61: 'accent', 62: 'accept', 63: 'acceptable', 64: 'accepted', 65: 'accepting', 66: 'acc

In [27]:
#Shuffling data serves the purpose of reducing variance and making sure that models remain general and overfit less.
#Shuffling the order in which examples are fed to the classifier is helpful so that batches between epochs do not look alike
#Doing so will eventually make our model more robust.
lines = shuffle(lines)
lines.head(10)

,source,target,comments
59073,i know how busy tom has been,START_ sé lo ocupado que ha estado tom _END,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
50669,i wish he were on our team,START_ desearía que él estuviera en nuestro eq...,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
110839,tom put down his fork and pushed his chair back,START_ tom dejó el tenedor y empujó su silla h...,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
112813,the place just doesnt look as good as it used to,START_ el lugar tan sólo no se ve tan bien com...,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
9696,tom saw a mouse,START_ tom miró un ratón _END,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
10388,do you live here,START_ ¿vivís aquí _END,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
71643,i wasnt there when it happened,START_ no estaba allí cuando pasó _END,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
44355,what else can i tell you,START_ ¿qué más te puedo decir _END,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
33413,id better wake tom up,START_ será mejor que despierte a tom _END,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
4325,i can explain,START_ yo puedo explicar _END,CC-BY 2.0 (France) Attribution: tatoeba.org #2...


In [28]:
# Train - Test Split
X, y = lines.source, lines.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train.shape, X_test.shape

((110642,), (12294,))

#### Save the train and test dataframes for reproducing the results later, as they are shuffled.

In [29]:
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')

In [30]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        #print(word)
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
                    
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

### Encoder - Decoder Model Architecture

In [31]:
latent_dim = 50

In [32]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

Instructions for updating:
Colocations handled automatically by placer.


In [33]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [34]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [35]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model.png', show_shapes=True)

from IPython.display import Image
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model.png', show_shapes=True)

In [36]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 50

In [37]:
import datetime
print(datetime.datetime.now())

2020-02-05 22:10:42.770935


In [38]:
#model.load_weights('nmt_weights_orig175.h5')

In [ ]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

### Always remember to save the weights

In [ ]:
model.save_weights('nmt_weights_nopad100.h5')

In [39]:
import datetime
print(datetime.datetime.now())

2020-02-05 13:04:55.210141


### Load the weights, if you close the application

In [38]:
model.load_weights('nmt_weights_orig185.h5')

### Inference Setup

In [42]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [43]:
plot_model(encoder_model, to_file='encoder_model.png', show_shapes=True)

In [44]:
plot_model(decoder_model, to_file='decoder_model.png', show_shapes=True)

### Decode sample sequeces

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

### Evaluation on Train Dataset

In [ ]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Source sentence:', X_train[k:k+1].values[0])
print('Actual Target Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Target Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Source sentence:', X_train[k:k+1].values[0])
print('Actual Target Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Target Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Source sentence:', X_train[k:k+1].values[0])
print('Actual Target Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Target Translation:', decoded_sentence[:-4])

In [ ]:
val_gen = generate_batch(X_test, y_test, batch_size = 1)
k=-1

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Target Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Spanish Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Target Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Spanish Translation:', decoded_sentence[:-4])